# Day 13: Knights of the Dinner Table

In [1]:
import re
from collections import defaultdict
from itertools import permutations

from tools import loader, parsers

DATA = parsers.lines(loader.get(2015, 13))

First, we need to parse our input. Regex makes it easy, we just need to follow the structure of the centenses:\
`(\w+).*(gain|lose)` captures the first word, skippes everything that follows until we reach the word 'gain' or 'lose'\
`(gain|lose) (\d+)` captures any number after a space\
`.*to (\w+)` captures the word after 'to '

Alternatively, we can achieve the same in a more verbose way: `(\w+) would (gain|lose) (\d+) happiness units by sitting next to (\w+).`

The only minor annoyance is that we need to convert 'gain' and 'lose' into + and - to construct a proper integer.

In [2]:
guests = defaultdict(dict)
for line in DATA:
    info = re.findall(r'(\w+).*(gain|lose) (\d+).*to (\w+)', line)[0]
    value = '-' if info[1] == 'lose' else '+'
    guests[info[0]][info[3]] = int(value + info[2])

As for the actual solution, it's bruteforce time! (again?) The amount of permutations is only ~40k for part 1 and ~363k for part 2. Since we don't need to perform any expensive computations, it all completes in a second or two.

We only need the right neighbour, whom we get by index (using modulo to wrap to index 0 when we reach the end). Once we have both neighbours, we simply look them up in our dictionary of guests. I use `.get()` with a default value of 0 for part 2 in order to avoid adding 'self': 0 to every guest in the guests dictionary.

An alternative approach could be precumputing happiness change between all pairs of guests to avoid repeating calculations, or caching the calculated happiness change, but the execution time is fairly low as is, so we won't see a significant difference.

In [3]:
def optimize_seating(part2: bool) -> int:
    if part2:
        guests['self'] = dict.fromkeys(guests, 0)
    best_happiness = 0
    for combo in permutations(guests.keys(), len(guests)):
        happiness = 0
        for i, guest in enumerate(combo):
            neighbour = combo[(i + 1) % len(combo)]
            happiness += guests[guest].get(neighbour, 0)
            happiness += guests[neighbour].get(guest, 0)
        best_happiness = max(best_happiness, happiness)
    return best_happiness


print(optimize_seating(part2=False))  # 733
print(optimize_seating(part2=True))  # 725

733
725
